# Loading Data

- DataSet 관리
- DataLoader

데이터가 많을 수록 학습할 표본이 많은것이다. 
하지만, 데이터가 많아질 수록 서버가 느려지며 전체를 load할 수 없을수도 있다. 

## Minibatch Gradient Descent

- 전체 데이터를 균일하게 나눠 학습시키는 방식
- 각 Minibatch만큼만 cost를 계산해 경사하강을 이용해 prediction을 계산한다.
- 각 업데이트마다 데이터의 수가 적어져 빠르다. 
- 전체를 쓰지 않아 cost를 그래프화시키면 지진계처럼 흔들려있다. 

In [2]:
from torch.utils.data import Dataset

class CustomDataSet(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]]
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y
    
dataset = CustomDataSet()

## Pytorch DataLoader

- 데이터 로더를 초기화하기 위해서, `Dataset`과 데이터셋을 분할하여 예측 계산하기 위한 `minibatch`의 크기를 입력해야 한다. 
- 순서대로 학습할 경우, 매번 학습 결과가 같아진다. 이를 위해 shuffle을 True로 가져간다.

In [3]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)

- enumerate
    - minibatch의 인덱스와 데이터를 받아 순회시킬 때 사용한다. 
- len(dataloader)
    - epoch당 minibatch의 개수를 알려준다. 

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultivariableLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        return self.linear(x)

model = MultivariableLinearRegression()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20

for epoch in range(nb_epochs+1):
    for batch_idx, sample in enumerate(dataloader):
        x_train, y_train = sample
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Hypothesis: {}, Cost: {:.6f}'.format(epoch,nb_epochs,prediction.squeeze().detach(),cost.item()))

Epoch    0/20 Hypothesis: tensor([93.1129, 70.0796]), Cost: 6360.955566
Epoch    0/20 Hypothesis: tensor([139.4846, 128.3395]), Cost: 3202.196289
Epoch    0/20 Hypothesis: 133.30250549316406, Cost: 349.596313
Epoch    1/20 Hypothesis: tensor([139.7924, 125.2383]), Cost: 214.989349
Epoch    1/20 Hypothesis: tensor([170.0131, 171.2277]), Cost: 144.706314
Epoch    1/20 Hypothesis: 191.8470458984375, Cost: 17.247028
Epoch    2/20 Hypothesis: tensor([178.0339, 194.2409]), Cost: 3.479938
Epoch    2/20 Hypothesis: tensor([136.5596, 152.1379]), Cost: 14.808642
Epoch    2/20 Hypothesis: 181.33746337890625, Cost: 13.414174
Epoch    3/20 Hypothesis: tensor([138.7309, 198.3184]), Cost: 8.030951
Epoch    3/20 Hypothesis: tensor([183.1740, 181.7793]), Cost: 3.250013
Epoch    3/20 Hypothesis: 154.5157928466797, Cost: 6.329214
Epoch    4/20 Hypothesis: tensor([197.2146, 182.1458]), Cost: 4.810948
Epoch    4/20 Hypothesis: tensor([153.9649, 138.2460]), Cost: 8.976678
Epoch    4/20 Hypothesis: 181.44903

## What's next?

- 숫자 하나의 답이 아닌, 분류를 위한 모델을 학습한다. 